In [1]:
# !pip install tensorflow_hub tensorflow_text

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

C:\Users\conta\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df=pd.read_csv("C:/Users/conta/Downloads/BERTspam.csv")

In [4]:
df.head(2)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [5]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [6]:
#Here the the ham emails are more than spam so we do down sampling
df_spam=df.loc[df['Category']=="spam"]

In [7]:
df_ham=df.loc[df['Category']=="ham"].sample(df_spam.shape[0])

In [8]:
#Now the shapes are equal so we concatenate this 2
print(df_spam.shape)
print(df_ham.shape)

(747, 2)
(747, 2)


In [9]:
df_balanced=pd.concat([df_spam,df_ham])

In [10]:
df_balanced

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
1085,ham,For me the love should start with attraction.i...
4615,ham,hows my favourite person today? r u workin har...
3373,ham,And now electricity just went out fml.
4860,ham,"Hey, a guy I know is breathing down my neck to..."


In [11]:
df_balanced["Category"].replace({"spam":1,"ham":0},inplace=True)

In [12]:
X=df_balanced["Message"]
y=df_balanced["Category"]

In [13]:
X

2       Free entry in 2 a wkly comp to win FA Cup fina...
5       FreeMsg Hey there darling it's been 3 week's n...
8       WINNER!! As a valued network customer you have...
9       Had your mobile 11 months or more? U R entitle...
11      SIX chances to win CASH! From 100 to 20,000 po...
                              ...                        
1085    For me the love should start with attraction.i...
4615    hows my favourite person today? r u workin har...
3373               And now electricity just went out fml.
4860    Hey, a guy I know is breathing down my neck to...
1359    If i start sending blackberry torch to nigeria...
Name: Message, Length: 1494, dtype: object

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=20,stratify=df_balanced["Category"])

In [15]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [16]:
#Just for testing we are writing this function can move forward to building the model
def get_sentence_embeddings(sentences):
    preprocessed_text=bert_preprocess(sentences)
    embedded_text=bert_encoder(preprocessed_text)
    return embedded_text["pooled_output"]

In [17]:
#Example
# get_sentence_embeddings(["Free entry in 2 a wkly comp to win FA Cup final","Not a lot has happened here. "])

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
from keras.layers import Dense,Dropout,Input
import keras
import tensorflow as tf

In [20]:
#Bert layers
text_input=Input(shape=(),dtype=tf.string)
preprocess_text=bert_preprocess(text_input)
outputs=bert_encoder(preprocess_text)

#Neural network layers
l=Dropout(0.1)(outputs["pooled_output"])
l2=Dense(units=1,activation="sigmoid")(l)

#Construct final model
model=keras.Model(inputs=[text_input],outputs=[l2])


In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [22]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [23]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5
35/35 [==============================] - 733s 21s/step - loss: 0.6523 - accuracy: 0.5982
Epoch 2/5
35/35 [==============================] - 718s 21s/step - loss: 0.5077 - accuracy: 0.8214
Epoch 3/5
35/35 [==============================] - 719s 21s/step - loss: 0.4309 - accuracy: 0.8652
Epoch 4/5
35/35 [==============================] - 715s 20s/step - loss: 0.3928 - accuracy: 0.8768
Epoch 5/5
35/35 [==============================] - 715s 20s/step - loss: 0.3624 - accuracy: 0.8875


In [24]:
model.predict(["Hello"])

1/1 [==============================] - 2s 2s/step


array([[0.07983191]], dtype=float32)

In [25]:
predictions=model.predict(X_test)

12/12 [==============================] - 240s 20s/step


In [26]:
import numpy as np
y_pred=np.where(predictions>0.5,1,0)

In [27]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91       187
           1       0.90      0.94      0.92       187

    accuracy                           0.91       374
   macro avg       0.92      0.91      0.91       374
weighted avg       0.92      0.91      0.91       374

